In [7]:
import csv
import cv2
import numpy as np

lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)

X_train = np.array(images)
y_train = np.array(measurements)


In [9]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=10)

model.save('model.h5')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1252 samples, validate on 313 samples
Epoch 1/10
  32/1252 [..............................] - ETA: 5s - loss: 42273.9609

 128/1252 [==>...........................] - ETA: 1s - loss: 142537844.9902

 256/1252 [=====>........................] - ETA: 0s - loss: 137133613.0605

 384/1252 [========>.....................] - ETA: 0s - loss: 121401446.7070

...

 704/1252 [===============>..............] - ETA: 0s - loss: 4404.3977

 864/1252 [===================>..........] - ETA: 0s - loss: 4382.0526

1024/1252 [=======================>......] - ETA: 0s - loss: 4387.5446

1184/1252 [===========================>..] - ETA: 0s - loss: 4330.7426

1252/1252 [==============================] - 0s - loss: 4238.4036 - val_loss: 3138.9087